In [1]:
#Take a look on this
#https://medium.com/@sebastiannorena/finding-correlation-between-many-variables-multidimensional-dataset-with-python-5deb3f39ffb3
    
import pandas as pd
import numpy as np
import nltk
import re
import os

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from zipfile import ZipFile


lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

nltk.download('stopwords')
nltk.download('wordnet')


TEXT_COL_NAME = 'Text'
LABEL_COL_NAME = 'Label'

TOXIC_LABEL = 'TOXIC'
HEALTHY_LABEL = 'HEALTHY'

THRESHOLD = 0.3


DEFAULT_DISTRIBUTION = (50000,50000)


def exportCSV(df,name,distribution=DEFAULT_DISTRIBUTION):
    dis_positive = distribution[0]
    dis_negative = distribution[1]
    
    ds_positive  = df[df[LABEL_COL_NAME] == TOXIC_LABEL]
    ds_negative  = df[df[LABEL_COL_NAME] == HEALTHY_LABEL]
    
    ds_positive_len = len(ds_positive)
    ds_negative_len = len(ds_negative)
    
    i_ds_positive = np.arange(ds_positive_len)
    np.random.shuffle(i_ds_positive)
    
    i_ds_negative = np.arange(ds_negative_len)
    np.random.shuffle(i_ds_negative)
    
    max_positives = ds_positive_len
    max_negatives = ds_negative_len
    
    if(dis_positive < max_positives):
        max_positives = dis_positive

    if(dis_negative < max_negatives):
        max_negatives = dis_negative
        
    i_ds_positive = i_ds_positive[:max_positives]
    i_ds_negative = i_ds_negative[:max_negatives]
    
    out_df_positives = ds_positive.iloc[i_ds_positive]
    out_df_negatives = ds_negative.iloc[i_ds_negative]
    
    
    print("Total of positives: ",ds_positive_len,"Total of negatives: ", ds_negative_len)
    print("Total of positives exported: ",max_positives,"Total of negatives exported: ", max_negatives)
    
    
    out_df = pd.concat([out_df_positives,out_df_negatives])
    out_df_len = len(out_df)
    i_out_df = np.arange(out_df_len)
    np.random.shuffle(i_out_df)
    out_df = out_df.iloc[i_out_df]

    csv_name = name+'.csv'
    zip_name = name+'.zip'
    
    try:
        os.remove(csv_name)
    except:
        print("Error while deleting file ", csv_name)
        
    try:
        os.remove(zip_name)
    except:
        print("Error while deleting file ", zip_name)
    
    out_df.to_csv(csv_name, index=False)
    
    zipObj = ZipFile(zip_name, 'w')
    zipObj.write(csv_name)
    zipObj.close()
    
def setLabels(df):
    positives = df[LABEL_COL_NAME] == 1
    negatives = df[LABEL_COL_NAME] == 0
    
    df.loc[positives, LABEL_COL_NAME] = TOXIC_LABEL
    df.loc[negatives, LABEL_COL_NAME] = HEALTHY_LABEL
    
    return df


[nltk_data] Downloading package stopwords to /Users/a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dataset = pd.read_csv("./dataset.csv")
validation_ds = pd.read_csv("./validation_dataset.csv")

In [3]:
dataset.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
dataset.describe()

,id,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,parent_id,article_id,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
count,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,405130.000000,405130.000000,405130.000000,...,1.026228e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06,1.804874e+06
mean,3.738434e+06,1.030173e-01,4.582099e-03,1.387721e-02,2.263571e-02,8.115273e-02,9.311271e-03,0.011964,0.003205,0.001884,...,3.722687e+06,2.813597e+05,2.779269e-01,4.420696e-02,1.091173e-01,2.446167e+00,5.843688e-01,6.605974e-03,1.439019e+00,8.784694e+00
std,2.445187e+06,1.970757e-01,2.286128e-02,6.460419e-02,7.873156e-02,1.760657e-01,4.942218e-02,0.087166,0.050193,0.026077,...,2.450261e+06,1.039293e+05,1.055313e+00,2.449359e-01,4.555363e-01,4.727924e+00,1.866589e+00,4.529782e-02,1.787041e+01,4.350086e+01
min,5.984800e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,6.100600e+04,2.006000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
25%,7.969752e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,7.960188e+05,1.601200e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
50%,5.223774e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,5.222993e+06,3.321260e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00
75%,5.769854e+06,1.666667e-01,0.000000e+00,0.000000e+00,0.000000e+00,9.090909e-02,0.000000e+00,0.000000,0.000000,0.000000,...,5.775758e+06,3.662370e+05,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00
max,6.334010e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,...,6.333965e+06,3.995410e+05,1.020000e+02,2.100000e+01,3.100000e+01,3.000000e+02,1.870000e+02,1.000000e+00,1.866000e+03,4.936000e+03


Como ai lab solo acepta dos columnas, texto y class entonces eliminaremos todas las columnas excepto comment_text y target

In [5]:
dataset2cols = dataset.loc[:, dataset.columns.intersection(['target','comment_text'])]

Cambiamos el orden de las columnas para que target esté después de comment text

In [6]:
dataset2cols = dataset2cols.reindex(columns=['comment_text','target'])

In [7]:
dataset2cols.head()

,comment_text,target
0,"This is so cool. It's like, 'would you want yo...",0.000000
1,Thank you!! This would make my life a lot less...,0.000000
2,This is such an urgent design problem; kudos t...,0.000000
3,Is this something I'll be able to install on m...,0.000000
4,haha you guys are a bunch of losers.,0.893617


Ahora como ai lab solo acepta valores discretos tendremos que especificar un treshold para definir nuestras dos clases: no ofensivo (0) / ofensivo (1)

In [8]:
dataset2cols[dataset2cols.target >= THRESHOLD]

,comment_text,target
4,haha you guys are a bunch of losers.,0.893617
5,ur a sh*tty comment.,0.666667
6,hahahahahahahahhha suck it.,0.457627
11,This is a great story. Man. I wonder if the pe...,0.440000
13,It's ridiculous that these guys are being call...,0.600000
...,...,...
1804852,Xi and his comrades must be smirking over Trum...,0.300000
1804856,Believing in God or not believing in God are p...,0.500000
1804857,"I take your point, but I think you're shooting...",0.700000
1804866,There's no whine like Alberta whine!,0.300000


In [9]:
dataset2cols.insert(2, LABEL_COL_NAME, np.zeros(len(dataset2cols)), True)

In [10]:
dataset2cols

,comment_text,target,Label
0,"This is so cool. It's like, 'would you want yo...",0.000000,0.0
1,Thank you!! This would make my life a lot less...,0.000000,0.0
2,This is such an urgent design problem; kudos t...,0.000000,0.0
3,Is this something I'll be able to install on m...,0.000000,0.0
4,haha you guys are a bunch of losers.,0.893617,0.0
...,...,...,...
1804869,"Maybe the tax on ""things"" would be collected w...",0.000000,0.0
1804870,What do you call people who STILL think the di...,0.000000,0.0
1804871,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.0
1804872,Anyone who is quoted as having the following e...,0.621212,0.0


In [11]:
dataset2cols[LABEL_COL_NAME].values[dataset2cols.target >= THRESHOLD] = 1

In [12]:
dataset2cols[dataset2cols.target >= THRESHOLD].describe()

,target,Label
count,266089.000000,266089.0
mean,0.517449,1.0
std,0.186804,0.0
min,0.300000,1.0
25%,0.400000,1.0
50%,0.500000,1.0
75%,0.644737,1.0
max,1.000000,1.0


In [13]:
finalDataset = dataset2cols.reindex(columns=['comment_text',LABEL_COL_NAME])
finalDataset = finalDataset.rename(columns={"comment_text": TEXT_COL_NAME})
finalDataset = setLabels(finalDataset)

In [14]:
finalDataset.head()

,Text,Label
0,"This is so cool. It's like, 'would you want yo...",HEALTHY
1,Thank you!! This would make my life a lot less...,HEALTHY
2,This is such an urgent design problem; kudos t...,HEALTHY
3,Is this something I'll be able to install on m...,HEALTHY
4,haha you guys are a bunch of losers.,TOXIC


In [15]:
exportCSV(finalDataset,'train_small_300',(300,300))
exportCSV(finalDataset,'train_small_3k',(3000,3000))
exportCSV(finalDataset,'train_small_30k',(30000,30000))
exportCSV(finalDataset,'train_small_50k',(50000,50000))
exportCSV(finalDataset,'train_small_100k',(100000,100000))

Total of positives:  266089 Total of negatives:  1538785
Total of positives exported:  300 Total of negatives exported:  300
Total of positives:  266089 Total of negatives:  1538785
Total of positives exported:  3000 Total of negatives exported:  3000
Total of positives:  266089 Total of negatives:  1538785
Total of positives exported:  30000 Total of negatives exported:  30000
Total of positives:  266089 Total of negatives:  1538785
Total of positives exported:  50000 Total of negatives exported:  50000
Total of positives:  266089 Total of negatives:  1538785
Total of positives exported:  100000 Total of negatives exported:  100000
Error while deleting file  train_small_100k.csv
Error while deleting file  train_small_100k.zip


Despues de tener nuestro dataset listo  y exportado vamos ahora a trabajar nuestro dataset de validacion que se encuentra acá https://github.com/t-davidson/hate-speech-and-offensive-language/tree/master/data

Si en el dataset de validacion la clase es 2 entonces es no ofensivo


In [16]:
validation_ds.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [17]:
validation_ds = validation_ds.loc[:, validation_ds.columns.intersection(['tweet','class'])]

In [18]:
validation_ds.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [19]:
validation_ds.insert(2, LABEL_COL_NAME, np.ones(len(validation_ds)), True)

In [20]:
validation_ds.head()

,class,tweet,Label
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,1.0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1.0
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1.0
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1.0
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1.0


In [21]:
validation_ds[LABEL_COL_NAME].values[validation_ds['class'] == 2] = 0

In [22]:
validation_ds.head()

,class,tweet,Label
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,0.0
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1.0
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1.0
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1.0
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1.0


In [23]:
validation_ds = validation_ds.rename(columns={"tweet": TEXT_COL_NAME})

In [24]:
validation_ds = validation_ds.loc[:, validation_ds.columns.intersection([TEXT_COL_NAME,LABEL_COL_NAME])]

In [25]:
validation_ds.head()

,Text,Label
0,!!! RT @mayasolovely: As a woman you shouldn't...,0.0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1.0
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1.0
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1.0
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1.0


Ahora necesitaremos limpiar nuestro texto de caracteres indeseados para eso usaremos una funcion implementada acá https://stackoverflow.com/questions/54396405/how-can-i-preprocess-nlp-text-lowercase-remove-special-characters-remove-numb


In [26]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    return " ".join(tokens)


In [27]:
validation_ds[TEXT_COL_NAME] = validation_ds[TEXT_COL_NAME].map(lambda s:preprocess(s)) 
validation_ds = setLabels(validation_ds)
validation_ds.head()

,Text,Label
0,rt mayasolovely as a woman you shouldn t compl...,HEALTHY
1,rt mleew boy dats cold tyga dwn bad for cuffin...,TOXIC
2,rt urkindofbrand dawg rt sbabylife you ever fu...,TOXIC
3,rt c_g_anderson viva_based she look like a tranny,TOXIC
4,rt shenikaroberts the shit you hear about me m...,TOXIC


In [28]:
exportCSV(validation_ds,'validation_small',(20000,4000))

Total of positives:  20620 Total of negatives:  4163
Total of positives exported:  20000 Total of negatives exported:  4000


BOW
TFIDF
FastText
DistilBERT
BERT
